## Lipas data fetching

In this script Lipas data is fetched from geoserver. Then the data is saved to a file. After this, the saved data is used to fetch Mapple Insights reachabilities from the POIs.

In [3]:
# import necessary modules
import pandas as pd
import geopandas as gpd
import geojson
import shapely
import shapely.wkt
import json
import os
%run scripts # getGrid, getLipasData, getReachabilityDF, visualize
import mapclassify

Choose your municipality. For example, type Helsinki or Jyväskylä. The Finnish name of the municipality is required.

In [3]:
municipality = input()

 Helsinki


In [4]:
# 1180 = frisbeegolf_rata
# 1340 = pallokenttä
# 2120 = kuntosali
# These will get data from lipas using typecode and typename
disc_golf_parks = getLipasData(typecode = '1180', 
                               typename = 'frisbeegolf_rata', 
                               municipality=municipality)
'''
fitness_centers = getLipasData(typecode = '2120', 
                               typename = 'kuntosali', 
                               municipality=municipality)
ball_parks = getLipasData(typecode = '1340', 
                          typename = 'pallokentta', 
                          municipality=municipality)
'''

"\nfitness_centers = getLipasData(typecode = '2120', \n                               typename = 'kuntosali', \n                               municipality=municipality)\nball_parks = getLipasData(typecode = '1340', \n                          typename = 'pallokentta', \n                          municipality=municipality)\n"

Subset the data to smaller size. Note that you do not have to execute this block for the program to work.

For example one disc golf course in Helsinki is in Santahamina garrison and is not available for the general public.

The column 'vapaa_kaytto' refers to whether the facility is publicly available. This however cannot be used for all sports facilties as there are multiple facilities with 'vapaa_kaytto' as None, but this does not mean that the facility is actually not accessible, it just lacks the information.

In [13]:
disc_golf_parks = disc_golf_parks.loc[disc_golf_parks['vapaa_kaytto'] == True]

Save Lipas data to geojson files. This needs to be done in order to
fetch Mapple Insights data for the layers.

Below is an example.

In [35]:
disc_golf_parks.to_file("./input_folder/hki_disc_golf.geojson", driver="GeoJSON")

If you have not already, do the following commands on your terminal. Remember that the folder should be mapple-api-script, or the folder where main.py is located at, when executing the last 3 commands. Note that if you have not created the output folder, the command creates a new folder with the same name.

conda activate mapple-insights-api

export MAPPLE_API_KEY=YOUR_MAPPLE_API_KEY

python main.py -u https://staging.api.mapple.io -e ./input_folder/hki_ball_parks.geojson -m 30 -d ./hki_ball_parks_output_folder -i True

python main.py -u https://staging.api.mapple.io -e ./input_folder/hki_disc_golf.geojson -m 30 -d ./hki_disc_golf_output_folder -i True

python main.py -u https://staging.api.mapple.io -e ./input_folder/hki_fitness_centers.geojson -m 30 -d ./hki_fitness_centers_output_folder -i True

I have set -i to True so I can construct the result layers more easily.

If you are unsure what do the -(letter) refer to, run "python main.py -h" on your terminal. This will give you the documentation.

Create 2 arrays that contain the names of the sports and traveling modes. And set the prefix accordingly. Prefix should be the same as in your output folder.

In [1]:
sports = ['disc_golf']
modes = ['walking', 'cycling', 'transit', 'driving']
prefix = input()

 hki_15


Assing population grid to a variable called grid.

In [ ]:
grid = getGrid()

This loop will create outputs as shapefiles in ./shapefiles-folder. The files are in folders named as prefix_sport_travelmode_grid.

In [5]:
for sport in sports:
    for mode in modes:
        print(r'Processing {}_{}_output_folder'.format(prefix,sport), end='\r')
        output = getReachabilityDF(r'{}_{}_output_folder'.format(prefix,sport), 
                                   r'{}'.format(mode))
        grid = grid.to_crs(output.crs)
        
        join = gpd.sjoin(output, grid, how="inner", op="intersects")
        join.to_file(r'./shapefiles/{}_{}_{}_grid'.format(prefix,sport,mode))

Now you should have reachability layers, as shapefiles, for the sport types you have chosen.